### Read chat.db into a csv

In [92]:
import sqlite3
import pandas as pd

# Connect to the chat.db file
conn = sqlite3.connect('../data/chat.db')

# Execute a query to fetch the data
cursor = conn.cursor()
cursor.execute('''
    SELECT chat.ROWID, message.date, handle.id, chat.room_name, message.text, message.is_from_me
    FROM
    (message INNER JOIN chat_message_join ON message.ROWID = chat_message_join.message_id
                INNER JOIN chat ON chat.ROWID = chat_message_join.chat_id
                    INNER JOIN chat_handle_join ON chat.ROWID = chat_handle_join.chat_id
                        INNER JOIN handle ON handle.ROWID = chat_handle_join.handle_id)
    ORDER BY chat.ROWID, message.date
''')

# Fetch all the rows
rows = cursor.fetchall()

# Create a DataFrame from the fetched rows
df = pd.DataFrame(rows, columns=['ROWID', 'date', 'handle_id', 'room_name', 'text', 'is_from_me'])

df_with_room_name = df[df['room_name'].notna()]
df_without_room_name = df[df['room_name'].isna()]

df_grouped = df_with_room_name.groupby(['text', 'room_name', 'ROWID', 'date', 'is_from_me'], as_index=False).agg({'handle_id': ', '.join})

df_final = pd.concat([df_grouped, df_without_room_name])

# Sort the final DataFrame
df_sorted = df_final.sort_values(['ROWID', 'date'])

# Save the DataFrame as a CSV file
df_sorted.to_csv('../data/chat_grouped.csv', index=False)

# Close the connection
conn.close()

# output looks like this: text, room_name, ROWID, date, is_from_me, handle_id

### CSV -> JSON for OpenAI Formatting

In [121]:
import datetime

# Create a datetime object for the start of 2021
dt = datetime.datetime(2023, 5, 1, tzinfo=datetime.timezone.utc)

# Convert to a timestamp in seconds
timestamp_s = dt.timestamp()

# Convert to nanoseconds
timestamp_ns = int(timestamp_s * 1e9)

print(timestamp_ns)

1682899200000000000


In [122]:
# Convert the 'date' column to int64
df['date'] = df['date'].astype('int64')
# print(df['date'].max())
# print(df['date'].min())

# Filter the DataFrame to only include messages sent after the timestamp. apple starts from 2001 instead of 1970 so we have to subtract
df = df[df['date'] > 1682899200000000000-978307200000000000]

In [123]:
# Reset the index of the DataFrame
df = df.reset_index(drop=True)

# Get the index of the filtered dataframe
index = df[df['is_from_me'] == 1].index

# Get the index of all messages right before the indices where 'is_from_me' is 1
preceding_index = index - 1

# Handle the case where index is 0
preceding_index = [i if i >= 0 else 0 for i in preceding_index]

# Get the rows corresponding to 'index'
df_index = df.loc[index]

# Get the rows corresponding to 'preceding_index'
df_preceding_index = df.loc[preceding_index]

# Concatenate the two DataFrames
df_final = pd.concat([df_preceding_index, df_index])

# Save the final DataFrame as a CSV file
df_final.to_csv('../data/chat_ready.csv', index=False)

In [124]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Get the SYS_PROMPT from the environment variables
sys_prompt = os.getenv('SYS_PROMPT')

In [125]:
import json

# Initialize an empty list to store the conversations
conversations = []

# Iterate over the DataFrame
for i in range(0, len(df_final), 2):
    # Initialize an empty list to store the messages for this conversation
    conversation = []

    # Get the two messages
    message_1 = df_final.iloc[i]
    message_2 = df_final.iloc[i+1]

    # Skip this pair if either message is None (tbh idk why this happens)
    if message_1['text'] is not None and message_2['text'] is not None:
        conversation.append({"role": "system", "content": sys_prompt})
        conversation.append({"role": "user", "content": message_1['text']})
        conversation.append({"role": "assistant", "content": message_2['text']})

        # Add the conversation to the conversations
        conversations.append({"messages": conversation})

# Convert the conversations to JSONL
with open('../data/conversations.jsonl', 'w') as f:
    for conversation in conversations:
        f.write(json.dumps(conversation) + '\n')

### Validate Formatting (If this doesn't validate, the OpenAI call will NOT work)

https://cookbook.openai.com/examples/chat_finetuning_data_prep

In [126]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

In [127]:
data_path = "../data/conversations.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 52787
First example:
{'role': 'system', 'content': "You are N, a 20 year old college student at Duke University. Respond to these texts in the diction and phrasing of a college student (casual). Be nice and concise using texting language. I usually text like this: if someone says 'hey', i'll say 'what's up'"}
{'role': 'user', 'content': 'If you don’t get upgraded to first class on your flight home, it’s only $45 to check a second luggage in.\xa0\xa0So you can still ship your golf bag home.'}
{'role': 'assistant', 'content': 'ok'}


In [128]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [129]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [130]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 79, 2182
mean / median: 96.46600488756701, 91.0
p5 / p95: 83.0, 111.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 1, 778
mean / median: 9.688161858033228, 7.0
p5 / p95: 2.0, 18.0

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


In [131]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~5092151 tokens that will be charged for during training
By default, you'll train for 1 epochs on this dataset
By default, you'll be charged for ~5092151 tokens


### Send to OpenAI for finetuning!

In [140]:
from dotenv import load_dotenv
import os
import openai

# Load the .env file
load_dotenv()

import os

# Get the OpenAI key
openai.api_key = os.getenv('OPENAI_KEY')

In [144]:
from openai import OpenAI
client = OpenAI()

client.files.create(
  file=open("../data/conversations.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-VCiHe0C5Yp5UmZKzMYILx3bT', bytes=24018188, created_at=1714574957, filename='conversations.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [145]:
client.fine_tuning.jobs.create(
  training_file="file-VCiHe0C5Yp5UmZKzMYILx3bT", 
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-TGDhzMFuMmjOY2YdyHvsjw3c', created_at=1714574962, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-cWqfKdW2JgeuqBfCv0QND0FU', result_files=[], seed=1835378914, status='validating_files', trained_tokens=None, training_file='file-VCiHe0C5Yp5UmZKzMYILx3bT', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)